In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    BatchNormalization,
    Input,
    LeakyReLU,
    Flatten,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis=-1)  # 增加通道维度
x_test = np.expand_dims(x_test, axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train)

# 构建多层感知机（MLP）神经网络模型
model = Sequential(
    [
        Flatten(input_shape=(28, 28, 1)),

        Dense(1024), # Dense 全连接层
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        
        # ------- 做两层完全一样的 强化提取特征值 -------
        
        Dense(512),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(64),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(32),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        # ------------------ 输出 ------------------
        Dense(10, activation="softmax"),  # 输出层
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),  # 使用RMSprop优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
# 在损失函数平稳时减小学习率
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)

# 提前停止 达到最优值时停止训练
early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True, verbose=1
)

# 训练模型
model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=1000,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 64,
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)  # 使用调整后的测试数据形状
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_mlp_ubuntu.keras")

2025-01-14 18:24:15.980864: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2025-01-14 18:24:17.630464: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-01-14 18:24:17.631671: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-01-14 18:24:17.638117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2025-01-14 18:24:17.638138: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2025-01-14 18:24:17.639797: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.

Epoch 1/1000


2025-01-14 18:24:20.277189: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


937/937 [==============================] - 32s 33ms/step - loss: 2.6757 - accuracy: 0.1336 - val_loss: 1.7803 - val_accuracy: 0.3081
Epoch 2/1000
937/937 [==============================] - 30s 32ms/step - loss: 1.9326 - accuracy: 0.2971 - val_loss: 1.1451 - val_accuracy: 0.5167
Epoch 3/1000
937/937 [==============================] - 30s 32ms/step - loss: 1.6896 - accuracy: 0.4004 - val_loss: 1.0227 - val_accuracy: 0.5950
Epoch 4/1000
937/937 [==============================] - 31s 33ms/step - loss: 1.5503 - accuracy: 0.4643 - val_loss: 0.9381 - val_accuracy: 0.6340
Epoch 5/1000
937/937 [==============================] - 31s 33ms/step - loss: 1.4646 - accuracy: 0.5052 - val_loss: 0.8117 - val_accuracy: 0.7278
Epoch 6/1000
937/937 [==============================] - 28s 29ms/step - loss: 1.4027 - accuracy: 0.5349 - val_loss: 0.7261 - val_accuracy: 0.7396
Epoch 7/1000
937/937 [==============================] - 33s 35ms/step - loss: 1.3220 - accuracy: 0.5726 - val_loss: 0.5520 - val_accuracy